# 1. Retriever

In [1]:
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.embeddings import BedrockEmbeddings 
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1")

model_kwargs = { 
    "max_tokens_to_sample": 1024, 
    "temperature": 0, 
    "top_p": 0.5, 
    "stop_sequences": ["Human:"]
}

default_model_id = "anthropic.claude-v2"
bedrock_base_kwargs = dict(model_id=default_model_id, model_kwargs= model_kwargs)


In [2]:
persist_directory = './streamlit/docs/chroma/'
vectordb = Chroma(persist_directory=persist_directory, embedding_function=bedrock_embeddings)

In [3]:
print(vectordb._collection.count())
question = "Como se compone el Congreso Nacional?"

results = vectordb.max_marginal_relevance_search(query=question, k=8)

238


# 2. Chatbot

In [4]:



from langchain.chat_models import BedrockChat
from langchain.llms.bedrock import Bedrock
llm = BedrockChat(**bedrock_base_kwargs)
llm = Bedrock(**bedrock_base_kwargs)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    return_source_documents=True,
    verbose=True, 
    retriever=vectordb.as_retriever(search_type = "mmr", search_kwargs={"k": 10})
)

In [5]:
inputs_pre = qa_chain.prep_inputs({"query": "existe paridad en la propuesta de constitución?"})

In [6]:
qa_chain

RetrievalQA(verbose=True, combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"), llm=Bedrock(client=<botocore.client.BedrockRuntime object at 0x11cb74b90>, model_id='anthropic.claude-v2', model_kwargs={'max_tokens_to_sample': 1024, 'temperature': 0, 'top_p': 0.5, 'stop_sequences': ['Human:']})), document_variable_name='context'), return_source_documents=True, retriever=VectorStoreRetriever(tags=['Chroma', 'BedrockEmbeddings'], vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x1199c1590>, search_type='mmr', search_kwargs={'k': 10}))

In [7]:
from langchain.prompts import PromptTemplate

promp_template = """
Responde a la siguiente pregunta tan preciso como sea posible empleando el contexto encerrado por ##, 
si la respuesta no esta contenida en el contexto o si el contexto esta vacio responde "No lo se".

##{context}##

Question:{question}
"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=promp_template)

qa_chain.combine_documents_chain.llm_chain.prompt =QA_CHAIN_PROMPT


In [8]:
result = qa_chain({"query": "Qué requisitos existen para ser Senador?"})
print(result['query'],result['result'])



> Entering new RetrievalQA chain...
Prompt:

 
Responde a la siguiente pregunta tan preciso como sea posible empleando el contexto encerrado por ##, 
si la respuesta no esta contenida en el contexto o si el contexto esta vacio responde "No lo se".

##Capitulo 4: congreso nacional
articulo 54
54
para ser elegido diputado o senador se requiere ser ciudadano con derecho 
a sufragio, tener cumplidos veintiún y treinta y cinco años de edad al día de la 
elección, respectivamente, haber cursado la enseñanza media o equivalente y tener 
residencia en la región a que pertenezca el territorio electoral correspondiente 
durante un plazo no inferior a dos años, contado hacia atrás desde el día de la 
elección.

Capitulo 4: congreso nacional
articulo 53
53
1. el senado se compone de miembros elegidos en votación directa por 
circunscripciones senatoriales, en consideración a las regiones del país, cada una de 
las cuales constituirá, a lo menos, una circunscripción. la ley electoral respectiva 


In [13]:
result = qa_chain({"query": "existe paridad en la propuesta de constitución?"})
print(result['query'],result['result'])
result = qa_chain({"query": "existe igualdad hombre mujer en la propuesta de constitución?"})
print(result['query'],result['result'])
result = qa_chain({"query": "Puedo ser presidente si tengo 16 años?"})
print(result['query'], result['result'])
result = qa_chain({"query": "Cuáles otros requisitos hay para ser presidente?"})
print(result['query'], result['result'])
result = qa_chain({"query": "Como se compone el Congreso Nacional?"})
print(result['query'],result['result'])
result = qa_chain({"query": "Qué requisitos existen para ser Senador?"})
print(result['query'],result['result'])





> Entering new RetrievalQA chain...


KeyboardInterrupt: 

# 3. with Memory

In [11]:
result['source_documents'][0]

Document(page_content='54\npara ser elegido diputado o senador se requiere ser ciudadano con derecho \na sufragio, tener cumplidos veintiún y treinta y cinco años de edad al día de la \nelección, respectivamente, haber cursado la enseñanza media o equivalente y tener \nresidencia en la región a que pertenezca el territorio electoral correspondiente \ndurante un plazo no inferior a dos años, contado hacia atrás desde el día de la \nelección.', metadata={'articulo': 54, 'capitulo': 4, 'nombre_capitulo': 'congreso nacional'})

In [37]:
from langchain.prompts import PromptTemplate
template = """
You are a constitutional law assitant who answers questions to users (chilean citizens) about the new constitutional draft to be voted in december 17th 2023.
(when asked for the source, provide the following link https://www.procesoconstitucional.cl/wp-content/uploads/2023/10/Texto-aprobado-Consejo-Constitucional_06.10.23.pdf)
Use the following pieces of context to answer the question at the end. Provide page number, article and letter if it is part of the answer. 
Don't use other sources of information outside that was provided as context. Don't user your prior knowledge about laws. Don't make up the answer using your intrinsic knowledge.
If you don't know the answer, please propose a new question rephrasing, don't try to make up an answer. 
If the user doesn't ask a specific question (like greetings, goodbyes or thanking you) just reply the casual conversation. 
Never take a stand about approving or rejecting the draft, invite the user to read the draft and make his/her own informed opinion.
Always validate at the end if the answer was helpful to the user.

{context}
Question: {question}
Assistant:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template)


In [47]:
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    ai_prefix="A",
    human_prefix="H"
)

memory = ConversationSummaryBufferMemory(
    llm=llm, max_token_limit=256,
    human_prefix = "H", ai_prefix= "A" # Prefijos d usuartio y asistente custom para la historia.
)


In [48]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(search_type = "mmr",  search_kwargs={"k": 10} ),
    #return_source_documents=True,
    memory = memory, 
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

In [69]:
memory.moving_summary_buffer
memory.buffer_as_str

AttributeError: 'ConversationSummaryBufferMemory' object has no attribute 'buffer_as_str'

In [70]:
qa_chain.prep_inputs({"query": "y para ser dipoutado?"})

{'query': 'y para ser dipoutado?',
 'history': 'System:  H preguntó cuáles son los requisitos para ser presidente en Chile. A respondió que según la Constitución de Chile, los requisitos son: tener nacionalidad chilena, tener al menos 35 años de edad, poseer las calidades necesarias para ser ciudadano con derecho a voto. Esto está establecido en el artículo 92 de la Constitución. Luego, H preguntó sobre los requisitos para ser senador. A respondió que según el artículo 55 de la Constitución, se requiere: ser ciudadano con derecho a sufragio, tener 35 años de edad, haber cursado enseñanza media y tener residencia en la región correspondiente por al menos 2 años antes de la elección. La edad mínima para senador también es 35 años.\nH: y para ser dipoutado?\nA:  Según el contexto provisto, para ser diputado en Chile se deben cumplir los siguientes requisitos:\n\n- Ser ciudadano con derecho a sufragio.\n- Haber cursado la enseñanza media o equivalente. \n- Tener 21 años de edad cumplidos.\

In [66]:
#result = qa_chain({"query": "y para ser dipoutado?"})
#print(result["result"])

In [171]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    ai_prefix= "A",
    human_prefix="H"
)

retriever=vectordb.as_retriever(search_type = "mmr", similarity_top_k=5, )
qa = ConversationalRetrievalChain.from_llm(
    llm,
    verbose=True,
    retriever=retriever,
    # condense_question_prompt=QA_CHAIN_PROMPT,
    #return_source_documents=True,
    #combine_docs_chain_kwargs={"prompt": QA_CHAIN_PROMPT},
    memory=memory,
)

In [21]:
memory.buffer

[]

In [172]:
template = """
You are a constitutional proffessor who answers questions to users (chilean citizens) about the new constitutional draft to be voted in december.
Use the following pieces of context to answer the question at the end. provide page number, article and letter if it is part of the answer.
If you don't know the answer, please propose a new question rephrasing, don't try to make up an answer.
Always validate at the end if the answer was helpful to the user.

{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)
qa.combine_docs_chain.llm_chain.prompt = QA_CHAIN_PROMPT

In [173]:
print(qa.question_generator.prompt.template)

Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:


In [174]:
print(qa.combine_docs_chain.llm_chain.prompt.template)


You are a constitutional proffessor who answers questions to users (chilean citizens) about the new constitutional draft to be voted in december.
Use the following pieces of context to answer the question at the end. provide page number, article and letter if it is part of the answer.
If you don't know the answer, please propose a new question rephrasing, don't try to make up an answer.
Always validate at the end if the answer was helpful to the user.

{context}
Question: {question}
Helpful Answer:


In [178]:
qa.run({"question": "se protege el derecho a la educación?"})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: se protege el derecho a la educación?
Assistant:  Sí, la nueva constitución protege el derecho a la educación. En particular:

- Artículo 24, letra a) establece la libertad de enseñanza, que comprende el derecho a abrir establecimientos educacionales y desarrollar proyectos e idearios educativos.

- Artículo 24, letra b) señala que la libertad de enseñanza existe para garantizar a las familias el derecho a educar a sus hijos y elegir el tipo de educación.  

- Artículo 24, letra i) establece que el Estado deberá sostener y coordinar una red pluralista de establecimientos educacionales de calidad en todos los niveles.

- Artículo 24, letra l) señala que es deber del Estado y comunidad educativa promover el desarrollo profesional de docentes.

Espero haber podido

/Users/ensamblador/Documents/nueva-constitucion-2023/.venv/lib/python3.11/site-packages/langchain/llms/bedrock.py:50: UserWarning: Error: Prompt must alternate between '

Human:' and '

Assistant:'. Received 

Human: Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: se protege el derecho a la educación?

Assistant:  Sí, la nueva constitución protege el derecho a la educación. En particular:

- Artículo 24, letra a) establece la libertad de enseñanza, que comprende el derecho a abrir establecimientos educacionales y desarrollar proyectos e idearios educativos.

- Artículo 24, letra b) señala que la libertad de enseñanza existe para garantizar a las familias el derecho a educar a sus hijos y elegir el tipo de educación.  

- Artículo 24, letra i) establece que el Estado deberá sostener y coordinar una red pluralista de establecimientos educacionales de calidad en todos l


> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are a constitutional proffessor who answers questions to users (chilean citizens) about the new constitutional draft to be voted in december.
Use the following pieces of context to answer the question at the end. provide page number, article and letter if it is part of the answer.
If you don't know the answer, please propose a new question rephrasing, don't try to make up an answer.
Always validate at the end if the answer was helpful to the user.

16 
  
Solo en virtud de una ley de quorum  calificado, y siempre que no signifique tal 
discriminación, se podrán autorizar determinados beneficios directos o indirectos en favor de 
algún sector, actividad o zona geográfica, o establecer gravámenes especiales que afecten a 
uno u otras. En el caso de las franquicias o beneficios  indirectos, la estimación del costo de 
estos deberá incluirse anualmente en la Ley

' Sí, la nueva constitución protege el derecho al emprendimiento.\n\nEn el artículo 16, letra 34, se establece la "libertad para adquirir el dominio de toda clase de bienes, excepto aquellos que la naturaleza ha hecho comunes a todas las personas o que deban pertenecer a la Nación toda y la ley lo declare así". Esto incluye la libertad para emprender negocios y adquirir bienes necesarios para ello. \n\nAsimismo, en el mismo artículo y letra, se indica que "una ley de quorum calificado, cuando así lo exija el interés nacional, podrá establecer limitaciones o requisitos para la adquisición del dominio de algunos bienes". Esto significa que el emprendimiento está protegido, a menos que exista una ley especial que establezca restricciones por el interés nacional.\n\nEspero haber respondido de manera útil su consulta sobre la protección del derecho al emprendimiento en la nueva constitución. Por favor, indíqueme si requiere alguna aclaración adicional.'

In [175]:
result = qa({"question": "se protege el derecho a la educación?"})
print (result['answer'])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are a constitutional proffessor who answers questions to users (chilean citizens) about the new constitutional draft to be voted in december.
Use the following pieces of context to answer the question at the end. provide page number, article and letter if it is part of the answer.
If you don't know the answer, please propose a new question rephrasing, don't try to make up an answer.
Always validate at the end if the answer was helpful to the user.

científica y tecnológica, la creación artística y la protección e incremento del 
patrimonio cultural de la Nación.  
 
l) Es deber del Estado y de toda comunidad educativa promover el desarrollo 
profesional y respeto de los docentes y asistentes de la educación.  
 
24. La libertad de enseñanza.  
 
a) La libertad de enseñanza comprende el derecho de las personas de abrir, organizar y 
mantener establecimientos educacionales, así 

In [176]:
result = qa({"question": "y al emprendimiento?"})
print (result['answer'])



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: se protege el derecho a la educación?
Assistant:  Sí, la nueva constitución protege el derecho a la educación. En particular:

- Artículo 24, letra a) establece la libertad de enseñanza, que comprende el derecho a abrir establecimientos educacionales y desarrollar proyectos e idearios educativos.

- Artículo 24, letra b) señala que la libertad de enseñanza existe para garantizar a las familias el derecho a educar a sus hijos y elegir el tipo de educación.  

- Artículo 24, letra i) establece que el Estado deberá sostener y coordinar una red pluralista de establecimientos educacionales de calidad en todos los niveles.

- Artículo 24, letra l) señala que es deber del Estado y comunidad educativa promover el desarrollo profesional de docentes.

Espero haber podido

/Users/ensamblador/Documents/nueva-constitucion-2023/.venv/lib/python3.11/site-packages/langchain/llms/bedrock.py:50: UserWarning: Error: Prompt must alternate between '

Human:' and '

Assistant:'. Received 

Human: Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: se protege el derecho a la educación?

Assistant:  Sí, la nueva constitución protege el derecho a la educación. En particular:

- Artículo 24, letra a) establece la libertad de enseñanza, que comprende el derecho a abrir establecimientos educacionales y desarrollar proyectos e idearios educativos.

- Artículo 24, letra b) señala que la libertad de enseñanza existe para garantizar a las familias el derecho a educar a sus hijos y elegir el tipo de educación.  

- Artículo 24, letra i) establece que el Estado deberá sostener y coordinar una red pluralista de establecimientos educacionales de calidad en todos l


> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are a constitutional proffessor who answers questions to users (chilean citizens) about the new constitutional draft to be voted in december.
Use the following pieces of context to answer the question at the end. provide page number, article and letter if it is part of the answer.
If you don't know the answer, please propose a new question rephrasing, don't try to make up an answer.
Always validate at the end if the answer was helpful to the user.

16 
  
Solo en virtud de una ley de quorum  calificado, y siempre que no signifique tal 
discriminación, se podrán autorizar determinados beneficios directos o indirectos en favor de 
algún sector, actividad o zona geográfica, o establecer gravámenes especiales que afecten a 
uno u otras. En el caso de las franquicias o beneficios  indirectos, la estimación del costo de 
estos deberá incluirse anualmente en la Ley

In [179]:
from langchain.chains.question_answering import load_qa_chain

In [ ]:
chain = load_qa_chain(BedrockChat, chain_type="stuff", memory=memory, verbose=True,)